# Quantum Computation - Week 4

&nbsp;

## Contents

&nbsp;
    
1. [Grover's algorithm for a combinatorial problem (continued)](#gsearch)

&nbsp;

2. [C<sup>n</sup>NOT decomposition](#decomp)

&nbsp;

3. [Easter Assignments](#easter)

#### Module imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# importing Qiskit
from qiskit import Aer, IBMQ
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, transpile

from qiskit.tools.visualization import plot_histogram

#IBMQ.load_account()

**Quickly simulate circuit**

In [ ]:
# Plot results
def show_results(D):
    # D is a dictionary with classical bits as keys and count as value
    # example: D = {'000': 497, '001': 527}
    plt.bar(range(len(D)), list(D.values()), align='center')
    plt.xticks(range(len(D)), list(D.keys()))
    plt.show()

# Execute circuit, display a histogram of the results
def execute_locally(qc, draw_circuit=False):
    # Compile and run the Quantum circuit on a simulator backend
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim)
    result_sim = job_sim.result()
    result_counts = result_sim.get_counts(qc)
    
    # Print the results
    print("simulation: \n\n", result_counts)
    show_results(result_counts)

## 1. Grover's algorithm for a combinatorial problem (continued)<a id='gsearch'></a>


### Exactly-1 3-SAT problem

The Exactly-1 3-SAT problem is best explained with the following concrete problem. Let us consider a Boolean function $f$ with three Boolean variables $x_1, x_2, x_3$ as below.

$$
f(x_1, x_2, x_3) = (x_1 \vee x_2 \vee \neg x_3) \wedge (\neg x_1 \vee \neg x_2 \vee \neg x_3) \wedge (\neg x_1 \vee x_2 \vee x_3) 
$$

In the above function, the terms on the right-hand side equation which are inside $()$ are called clauses. Each clause has exactly three literals. Namely, the first clause has $x_1$, $x_2$ and $\neg x_3$ as its literals. The symbol $\neg$ is the Boolean NOT that negates (or, flips) the value of its succeeding literal. The symbols $\vee$ and $\wedge$ are, respectively, the Boolean OR and AND. The Boolean $f$ is satisfiable if there is an assignment of $x_1, x_2, x_3$ that evaluates to $f(x_1, x_2, x_3) = 1$ (or, $f$ evaluates to True). The Exactly-1 3-SAT problem requires us to find an assignment such that $f = 1$ (or, True) and there is *exactly* one literal that evaluates to True in every clause of $f$. 

A naive way to find such an assignment is by trying every possible combinations of input values of $f$. Below is the table obtained from trying all possible combinations of $x_1, x_2, x_3$. For ease of explanation, we interchangably use $0$ and False, as well as $1$ and True.  

|$x_1$ | $x_2$ | $x_3$ | $f$ | Comment | 
|------|-------|-------|-----|---------|
| 0    |  0    |  0    |  1  | Not a solution because there are three True literals in the second clause     | 
| 0    |  0    |  1    |  0  | Not a solution because $f$ is False          | 
| 0    |  1    |  0    |  1  | Not a solution because there are two True literals in the first clause        | 
| 0    |  1    |  1    |  1  | Not a solution because there are three True literals in the third clause        | 
| 1    |  0    |  0    |  0  | Not a solution because $f$ is False        | 
| 1    |  0    |  1    |  1  | **Solution**. BINGO!!       | 
| 1    |  1    |  0    |  1  | Not a solution because there are three True literals in the first clause        | 
| 1    |  1    |  1    |  0  | Not a solution because $f$ is False        | 


From the table above, we can see that the assignment $x_1x_2x_3 = 101$ is the solution for the Exactly-1 3-SAT problem to $f$. In general, the Boolean function $f$ can have many clauses and more Boolean variables. 

&nbsp;

### Grover's algorithm - circuit

<img src="img/Grovers_algorithm.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="600 px" align="center"> 

Where $N = 2^n$ is the number of possible inputs.


<div class="alert alert-block alert-info">

**State Initialization**

<img src="img/init.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="400 px" align="center"> 

</div>

In [ ]:
# Inputs: QuantumCircuit circuit, input register f_in, output register f_out
def input_state(circuit, f_in, f_out):
    circuit.h(f_in)
    circuit.x(f_out)
    circuit.h(f_out)

<div class="alert alert-block alert-info">

**Blackbox function to check the assignment of Exactly-1 3-SAT problem**

&nbsp;

For each clause of $f$, we design a sub-circuit that outputs True if and only if there is exactly one True literal in the clause. Combining all sub-circuits for all clauses, we can then obtain the blackbox that outputs True if and only if all clauses are satisfied with exactly one True literal each.   

For example, let us consider the clause $ y = (x_1 \vee \neg x_2 \vee x_3)$. It is easy to see that $y$ defined as 

$$
y = x_1 \oplus \neg x_2 \oplus x_3 \oplus ( x_1 \wedge \neg x_2 \wedge x_3), 
$$

Note that this clause can be written as [1, -2, 3].


&nbsp;


**`sub_oracle`**

&nbsp;

This function checks the output of a single clause. E.g. for [1, -2, 3]:

&nbsp;

<img src="img/black_box1.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="400 px" align="center"> 

&nbsp;

It uses one ancilla qubit to decompose the $C^3 NOT$ gate.

&nbsp;

**`blackbox_u_f`**

&nbsp;

This function calls `sub_oracle` as many times as there are clausules in a formula; it also checks if every clause evaluates to `True` - and therefore uses as many ancilla qubits as there are clauses.

</div>


In [ ]:
# Inputs:
# quantum circuit
# input register f_in
# output QUBIT f_outx
# auxiliary register aux
# clausule (list of 3 integers)

def sub_oracle(circuit, f_in, f_outx, aux, clausule):
    for i in clausule:
        if i<0:
            circuit.x(f_in[abs(i)-1])
    
    for i in clausule:
        circuit.cx(f_in[abs(i)-1],f_outx)
        
    circuit.ccx(f_in[0], f_in[1], aux)
    circuit.ccx(f_in[2], aux , f_outx)
    circuit.ccx(f_in[0], f_in[1], aux)
    
    for i in clausule:
        if i<0:
            circuit.x(f_in[abs(i)-1])

<div class="alert alert-block alert-warning">

We can check if the subroutine works as intended by printing the circuit for a clause:

</div>

In [ ]:
f_in = QuantumRegister(3)
f_out = QuantumRegister(1)
aux = QuantumRegister(1)
qc = QuantumCircuit(f_in, aux, f_out)
clausula = [1,-2,3]

sub_oracle(qc, f_in, f_out[0], aux, clausula)
#input_state(qc, f_in, f_out)
qc.draw(output='mpl')

In [ ]:
# Inputs:
# quantum circuit
# input register f_in
# output register f_out
# auxiliary register aux
# formula (list of clausules)

def black_box_u_f(circuit, f_in, f_out, aux, sat_formula):
    for (n, clausule) in enumerate(sat_formula):
        sub_oracle(circuit, f_in, f_out[n], aux, clausule)
        circuit.barrier()
    
    # DECOMPOSITION OF C3NOT SINCE WE HAVE 3 CLAUSULES
    circuit.ccx(f_out[0], f_out[1], aux[0])
    circuit.ccx(f_out[2], aux[0], f_out[len(sat_formula)])
    circuit.ccx(f_out[0], f_out[1], aux[0])
    
    circuit.barrier()
    
    # RETURN REGISTERS TO ORIGINAL STATE (for next iterations)
    for (n, clausule) in enumerate(sat_formula):
        sub_oracle(circuit, f_in, f_out[n], aux, clausule)
        circuit.barrier()

<div class="alert alert-block alert-warning">

We can check that the oracle works, for the formula above (f should return 1 for x<sub>1</sub> = 1, x<sub>2</sub> = 0, x<sub>3</sub> = 1).

</div>

In [ ]:
sat_formula = [[1,2,-3],[-1,-2,-3],[-1,2,3]]

#3 input qubits (3 literals in each clause)
f_in = QuantumRegister(3)

#1 ancilla qubit (ccx decomposition)
aux = QuantumRegister(1)

#output qubits = number of clauses + 1 (final output)
f_out = QuantumRegister(4)

# Create circuit
qc = QuantumCircuit(f_in, aux, f_out)

# Prepare input states, x1=1, x2= 0, x3=1
qc.x(f_in[0])
qc.x(f_in[2])

qc.barrier()

black_box_u_f(qc, f_in, f_out, aux, sat_formula)

cr = ClassicalRegister(7)
qc.add_register(cr)

qc.measure(f_in[0], cr[0])
qc.measure(f_in[1], cr[1])
qc.measure(f_in[2], cr[2])

qc.measure(f_out[0], cr[3])
qc.measure(f_out[1], cr[4])
qc.measure(f_out[2], cr[5])
qc.measure(f_out[3],cr[6])

qc.draw(output='mpl')

In [ ]:
execute_locally(qc)

<div class="alert alert-block alert-info">

**Inversion about the mean**

In this step, we apply

&nbsp;

$$
H^{\otimes n} \left(2|0\rangle \langle 0 | - I \right) H^{\otimes n}
$$

&nbsp;

The function `inversion_about_mean` makes use of a $C^nZ$ gate, which is equivalent to a $C^nNOT$ gate with the target qubit encapsulated between Hadamard gates.

</div>

In [ ]:
# 3-Sat: at most 3 literals in each clause
def inversion_about_mean(circuit, f_in):
    n = f_in.__len__()
    
    circuit.h(f_in)
    circuit.x(f_in)
    
    
    circuit.h(f_in[n-1])
    
    if n == 2:
        circuit.cx(f_in[0], f_in[1])
    
    elif n == 3:
        circuit.ccx(f_in[0], f_in[1], f_in[2])
        
    circuit.h(f_in[n-1])
        
    circuit.x(f_in)
    circuit.h(f_in)

<div class="alert alert-block alert-warning">

Here is a circuit of the inversion about the mean on three qubits.

</div>

In [ ]:
qr = QuantumRegister(3)
qInvAvg = QuantumCircuit(qr)

inversion_about_mean(qInvAvg, qr)

qInvAvg.draw(output='mpl')

<div class="alert alert-block alert-warning">

**Exercise 4.1 - Putting it all together**

&nbsp;

<img src="img/Grovers_algorithm.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="600 px" align="center"> 

&nbsp;

The complete steps of Grover search are as follows.

1. Create the superposition of all possible solutions as the initial state (with working qubits initialized to zero)

$$  \sum_{j=0}^{2^{n}-1} \frac{1}{2^n} |j\rangle |0\rangle$$

2. Repeat for $T = \sqrt{N}$ times:

     * Apply the `blackbox` function
    
     * Apply the `inversion-about-the-mean` function
    
3. Measure `f_in` to obtain the solution

Note: $N = 2^n$, where $n$ is the number of qubits in `f_in`.

</div>

In [ ]:
def grover_3sat(sat_formula):
    # Number of clauses 
    k = 3
    
    # 3 literals - 3 qubits
    f_in = QuantumRegister(3, 'f_in')
    
    # Number of iterations: sqrt(2^3) - 2 iterations
    iterations = 2
    
    # Number of qubits in f_out: number of clauses + 1
    f_out = QuantumRegister(k+1, 'f_out')
    
    # Ancilla qubit
    aux = QuantumRegister(1, 'aux')
    
    # Measure f_in at the end to a classical register
    cr = ClassicalRegister(3)
    
    # Create Circuit
    grover = QuantumCircuit(f_in, aux, f_out, cr)
    

    


In [ ]:
sat_formula = [[1, 2, -3], [-1, -2, -3], [-1, 2, 3]]
grover_3sat(sat_formula)

&nbsp;

## 2. Grover search in higher-dimensions: C<sup>n</sup>NOT decomposition<a id='decomp'></a>

&nbsp;

What happens if we try to run the function for a larger number of variables (i.e. input qubits), or a larger number of clauses (i.e. f_out qubits)?

You will find that the algorithm will have to apply multiple C<sup>n</sup>NOTs, with n being the number of controls necessary to perform a NOT operation on the target qubit. This means not only larger f_in and f_out, but also more auxiliary qubits (for decomposition).

Admit the following decomposition scheme for a C<sup>n</sup>NOT gate (reference [1]):

<img src="img/decompose3.png" width="500 px" align="center">


<div class="alert alert-block alert-warning">

**Exercise 4.2 - C<sup>n</sup>NOT decomposition**

&nbsp;

From the scheme above, build a function `cn_not` to decompose a C<sup>n</sup>NOT with an arbitrary number of controls into Toffoli and CNOT gates. 

The function should take as arguments the quantum circuit, **a list of control qubits** (this is different from a qubit register), a target qubit, and a register of auxiliary qubits.

</div>

In [ ]:
def cn_not(circuit, ctrls, trg, aux):
    


<div class="alert alert-block alert-warning">

We can test the function - let's admit that controls are contained in a single quantum register:
    
</div>

In [ ]:
# Number of controls
n=4

controls=QuantumRegister(n, 'controls')
aux=QuantumRegister(n-1, 'aux')
trg=QuantumRegister(1, 'target')
qc=QuantumCircuit(controls, aux, trg)

control_list = []
for i in range(n):
    control_list.append(controls[i])
    
cn_not(qc, control_list, trg, aux)

qc.draw(output='mpl')


## 3. Easter Assignments<a id='easter'></a>

Let's randomly atribute assignments:

In [ ]:
# '00' - Assignment A, '01' - Assignment B, '11' - Assignment C
assignment = {'00' : 'A', '01' : 'B', '11' : 'C'}
init_state = np.array([1, 1, 0, 1]) * 1/np.sqrt(3)

qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc = QuantumCircuit(qr, cr)

qc.initialize(init_state, [qr[0], qr[1]])
qc.measure(qr, cr)

In [ ]:
# Assignee name
assignee = 'NAME'

# Simulate a single execution
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1)
qc_state = job.result().get_counts(qc)

print(qc_state)

state_string = list(qc_state.keys())[0]

print(assignee + ' gets assignment ' + assignment[state_string])

<div class="alert alert-block alert-warning">

**Assignment A**

&nbsp;

Extend the `grover_3sat` function with `cn_not` to allow for $ n> 3$ literals per clause; assume at most 3 clauses for the formula.

&nbsp;

Notes:
   * the register `f_in` has $n$ qubits for $n$ literals $x_i$;
   * the number of qubits in the `aux` register will also be affected;
   * besides `grover_3sat`, functions `sub_oracle` and `inversion_about_mean` need to be extended.

</div>

**Assignees**
   * 
   * 
   * 

<div class="alert alert-block alert-warning">

**Assignment B**

&nbsp;

Extend the `grover_3sat` function with `cn_not` to allow for $ k > 3$ _clausules per formula;_ assume at most 3 literals per clausule.

&nbsp;

Notes:
   * the register `f_out` has $k+1$ qubits for $k$ clausules;
   * the number of qubits in the `aux` register will also be affected;
   * besides `grover_3sat`, function `black_box_u_f` needs to be extended.

</div>

**Assignees**
   * 
   * 
   * 

<div class="alert alert-block alert-warning">

**Assignment C**

Implement a so-called _splitting decomposition_ of a $C^nNOT$ gate, which decomposes the $n$ controls into Toffoli gates in a recursive manner, requiring $n-2$ aux qubits:

<img src="img/decompose1.png" width="300 px" align="center">

In the image, a $C^7NOT$ gate is decomposed into $C^4NOT$ gates, which further need to be decomposed - for a $C^4NOT$ gate, the remaining decomposition would be:

<img src="img/decompose2.png" width="500 px" align="center">

Note that each stage of decomposition requires one aditional aux qubit.

</div>

**Assignees**
   * 
   * 
   * 

In [ ]:
def recursive_cn_not(circuit, ctrls, trg, aux):

## References

[1] "[Flexible Representation and Manipulation of Audio Signals on Quantum Computers](https://www.researchgate.net/publication/312091720_Flexible_Representation_and_Manipulation_of_Audio_Signals_on_Quantum_Computers)"